In [ ]:
# https://towardsdatascience.com/hands-on-generative-adversarial-networks-gan-for-signal-processing-with-python-ff5b8d78bd28

In [15]:
import time
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
from matplotlib import pyplot
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

In [3]:
# prepare real data
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6c4b9636-1bc4-40d7-988f-4598ab74dafd;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 104ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/15 16:19:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/15 16:19:38 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [4]:
def read_motifs():
    data = spark.read.parquet(*["s3a://sapient-bucket-trusted/prod/graph/motifs/filtered/*"]).cache()
    # about 13 minutes to read
    start_time = time.time()
    print(data.count())
    print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and cache time: %s seconds ---" % (time.time() - start_time))

In [5]:
def write_malicious(ds):
    ds_mal = ds.filter(col("malicious") == 1).cache()
    ds_mal.count()
    ds_mal.write.option("maxRecordsPerFile", 300000).mode("overwrite").partitionBy("motif").parquet("s3a://sapient-bucket-trusted/prod/graph/motifs/malicious/")
    ds.unpersist()
    ds_mal.unpersist()

In [28]:
def create_plot_data():
    data = spark.read.parquet(*["s3a://sapient-bucket-trusted/prod/graph/motifs/malicious/*"]).cache()
    start_time = time.time()
    print(data.count())
    print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and cache time: %s seconds ---" % (time.time() - start_time))
    # create duplicate data and swap columns
    data2 = data.cache()
    data2.count()
    data2 = data2.withColumnRenamed("src", "temp")
    data2 = data2.withColumnRenamed("dst", "src")
    data2 = data2.withColumnRenamed("temp", "dst")
    plot_data = data.union(data2).cache()
    plot_data.count()
    hrWindow = Window.partitionBy(hour("timestamp")).orderBy("timestamp")
    traceWindow = Window.partitionBy().orderBy("Trace")
    plot_data = plot_data.withColumn("x_val", row_number().over(hrWindow)).cache()
    plot_data = plot_data.withColumn("y_val", row_number().over(traceWindow)).cache()
    plot_data.count()
    plot_data.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet("s3a://sapient-bucket-trusted/prod/graph/motifs/plot/")
    plot_data.unpersist()
    data.unpersist()
    data2.unpersist()

In [56]:
dm = read_motifs()

1876866963
 7:01PM UTC on Apr 15, 2023 --- read and cache time: 809.0210809707642 seconds ---


In [57]:
df.groupBy( col('even') ).agg( count((col('event')))).orderBy( col("count(event)").desc() ).limit(15).toPandas()

NameError: name 'df' is not defined

In [31]:
# write_malicious(data)

In [32]:
# create_plot_data()

In [26]:
plot_data = spark.read.parquet("s3a://sapient-bucket-trusted/prod/graph/motifs/plot/*").cache()

In [27]:
start_time = time.time()
print(plot_data.count())
print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and cache time: %s seconds ---" % (time.time() - start_time))

4587520
 4:51PM UTC on Apr 15, 2023 --- read and cache time: 4.176474571228027 seconds ---


In [35]:
plot_data = plot_data.orderBy( col('Trace').asc()).limit(300000).cache()

In [36]:
plot_data.count()

300000

In [38]:
plot_data.write.option("header",True).mode("overwrite").csv("s3a://sapient-bucket-trusted/prod/graph/motifs/plot_csv/")

In [55]:
plot_data.groupBy( col('Trace') ).agg( countDistinct((col('hostname')))).orderBy( col("count(hostname)").desc() ).limit(15).toPandas()

,Trace,count(hostname)
0,1005022455044-2,1
1,1005022456260-2,1
2,1005022456264-2,1
3,1005022456267-2,1
4,1005022456272-2,1
5,1005022456276-2,1
6,1005022456277-2,1
7,1005022456278-2,1
8,1005022456281-2,1
9,1005022456282-2,1


In [48]:
plot_data.filter( col('event') == 'e2').limit(4).toPandas()

,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,...,file_path,direction,logon_id,requesting_domain,requesting_user,malicious,file_path_ext,timestamp_bins,x_val,y_val
0,1005022798242-3,e2,9a017fd4-947f-4890-af69-adce16d81e74,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:40:20.828,PROCESS,OPEN,SysClient0201.systemia.com,None,None,...,None,None,None,None,None,1,None,1.0,95504,31281
1,1005022798242-3,e2,9a017fd4-947f-4890-af69-adce16d81e74,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:40:20.828,PROCESS,OPEN,SysClient0201.systemia.com,None,None,...,None,None,None,None,None,1,None,1.0,95624,31282
2,1005022798243-3,e2,9a017fd4-947f-4890-af69-adce16d81e74,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:40:20.828,PROCESS,OPEN,SysClient0201.systemia.com,None,None,...,None,None,None,None,None,1,None,1.0,95484,31283
3,1005022798243-3,e2,9a017fd4-947f-4890-af69-adce16d81e74,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:40:20.828,PROCESS,OPEN,SysClient0201.systemia.com,None,None,...,None,None,None,None,None,1,None,1.0,95604,31284


In [47]:
plot_data.filter( col('Trace') == '1005022567347-5').toPandas()

,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,...,file_path,direction,logon_id,requesting_domain,requesting_user,malicious,file_path_ext,timestamp_bins,x_val,y_val
0,1005022567347-5,e1,78039857-c291-4da1-b4bf-813dd445f1b7,96913629-c1c9-4503-9586-4a91de0e7311,2019-09-23 15:23:55.877,FILE,READ,SysClient0201.systemia.com,None,None,...,\Device\HarddiskVolume1\Windows\Prefetch\POWER...,None,None,None,None,1,.pf,NaN,28,12515
1,1005022567347-5,e1,78039857-c291-4da1-b4bf-813dd445f1b7,96913629-c1c9-4503-9586-4a91de0e7311,2019-09-23 15:23:55.877,FILE,READ,SysClient0201.systemia.com,None,None,...,\Device\HarddiskVolume1\Windows\Prefetch\POWER...,None,None,None,None,1,.pf,NaN,104,12516
2,1005022567347-5,e4,1404b405-e255-488d-ae49-069307de36a4,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:40:27.474,PROCESS,OPEN,SysClient0201.systemia.com,None,None,...,None,None,None,None,None,1,None,0.0,251795,12517
3,1005022567347-5,e4,1404b405-e255-488d-ae49-069307de36a4,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 15:40:27.474,PROCESS,OPEN,SysClient0201.systemia.com,None,None,...,None,None,None,None,None,1,None,0.0,273115,12518
